# Dynamic programming: Value and Policy Iteration

### Loading Libraries

In [11]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from mdptoolbox import mdp
from itertools import product

# Time
import time
from time import process_time

### Setting Gridworld Up 

#### States, Actions & Rewards

In [14]:
grid_size = (3, 4)

blocked_cell = (1, 1)

baseline_reward = -0.02

absorbing_cells = {(0, 3): 1, (1, 3): -1}

In [16]:
actions = ['L', 'U', 'R', 'D']

num_actions = len(actions)
probs = [.1, .8, .1, 0]

In [36]:
to_1d = lambda x: np.ravel_multi_index(x, grid_size)

to_2d = lambda x: np.unravel_index(x, grid_size)

In [38]:
num_states = np.product(grid_size)

cells = list(np.ndindex(grid_size))

states = list(range(len(cells)))

In [40]:
cell_state = dict(zip(cells, states))

state_cell= dict(zip(states, cells))

In [42]:
absorbing_states = {to_1d(s):r for s, r in absorbing_cells.items()}

blocked_state = to_1d(blocked_cell)

In [44]:
state_rewards = np.full(num_states, baseline_reward)

state_rewards[blocked_state] = 0

for state, reward in absorbing_states.items():
    state_rewards[state] = reward

In [46]:
action_outcomes = {}

for i, action in enumerate(actions):
    probs_ = dict(zip([actions[j % 4] for j in range(i, num_actions + i)], probs))
    action_outcomes[actions[(i + 1) % 4]] = probs_

In [48]:
action_outcomes

{'U': {'L': 0.1, 'U': 0.8, 'R': 0.1, 'D': 0},
 'R': {'U': 0.1, 'R': 0.8, 'D': 0.1, 'L': 0},
 'D': {'R': 0.1, 'D': 0.8, 'L': 0.1, 'U': 0},
 'L': {'D': 0.1, 'L': 0.8, 'U': 0.1, 'R': 0}}

### Transition Matrix

In [51]:
def get_new_cell(state, move):
    cell = to_2d(state)
    if actions[move] == 'U':
        return cell[0] - 1, cell[1]
    elif actions[move] == 'D':
        return cell[0] + 1, cell[1]
    elif actions[move] == 'R':
        return cell[0], cell[1] + 1
    elif actions[move] == 'L':
        return cell[0], cell[1] - 1

In [53]:
state_rewards

array([-0.02, -0.02, -0.02,  1.  , -0.02,  0.  , -0.02, -1.  , -0.02,
       -0.02, -0.02, -0.02])

In [55]:
def update_transitions_and_rewards(state, action, outcome):
    if state in absorbing_states.keys() or state == blocked_state:
        transitions[action, state, state] = 1
    else:
        new_cell = get_new_cell(state, outcome)
        p = action_outcomes[actions[action]][actions[outcome]]
        if new_cell not in cells or new_cell == blocked_cell:
            transitions[action, state, state] += p
            rewards[action, state, state] = baseline_reward
        else:
            new_state= to_1d(new_cell)
            transitions[action, state, new_state] = p
            rewards[action, state, new_state] = state_rewards[new_state]

In [57]:
rewards = np.zeros(shape=(num_actions, num_states, num_states))

transitions = np.zeros((num_actions, num_states, num_states))

actions_ = list(range(num_actions))

for action, outcome, state in product(actions_, actions_, states):
    update_transitions_and_rewards(state, action, outcome)

In [59]:
rewards.shape, transitions.shape

((4, 12, 12), (4, 12, 12))

### PyMDPToolbox

#### Value Iteration

In [68]:
gamma = .99

epsilon = 1e-5

In [70]:
vi = mdp.ValueIteration(transitions=transitions,
                        reward=rewards,
                        discount=gamma,
                        epsilon=epsilon)

vi.run()
f'# Iterations: {vi.iter:,d} | Time: {vi.time:.4f}'

'# Iterations: 31 | Time: 0.0022'

In [72]:
policy = np.asarray([actions[i] for i in vi.policy])

pd.DataFrame(policy.reshape(grid_size))

,0,1,2,3
0,R,R,R,L
1,U,L,U,L
2,U,L,L,L


In [74]:
value = np.asarray(vi.V).reshape(grid_size)

pd.DataFrame(value)

,0,1,2,3
0,0.884143,0.925054,0.961986,0.000000
1,0.848181,0.000000,0.714643,0.000000
2,0.808345,0.773328,0.736099,0.516083


#### Policy Iteration

In [77]:
pi = mdp.PolicyIteration(transitions=transitions,
                        reward=rewards,
                        discount=gamma,
                        max_iter=1000)

pi.run()
f'# Iterations: {pi.iter:,d} | Time: {pi.time:.4f}'

'# Iterations: 7 | Time: 0.0171'

In [79]:
policy = np.asarray([actions[i] for i in pi.policy])

pd.DataFrame(policy.reshape(grid_size))

,0,1,2,3
0,R,R,R,L
1,U,L,U,L
2,U,L,L,L


In [81]:
value = np.asarray(pi.V).reshape(grid_size)

pd.DataFrame(value)

,0,1,2,3
0,0.884143,0.925054,0.961986,-1.389785e-16
1,0.848181,0.000000,0.714643,5.749281e-16
2,0.808345,0.773328,0.736099,5.160828e-01


#### Value Iteration

In [91]:
skip_states = list(absorbing_states.keys())+[blocked_state]

states_to_update = [s for s in states if s not in skip_states]

In [93]:
V = np.random.rand(num_states)

V[skip_states] = 0

In [95]:
gamma = .99

epsilon = 1e-5

In [97]:
iterations = 0

start = process_time()

converged = False

while not converged:
    V_ = np.copy(V)
    for state in states_to_update:
        q_sa = np.sum(transitions[:, state] * (rewards[:, state] + gamma* V), axis=1)
        V[state] = np.max(q_sa)
    if np.sum(np.fabs(V - V_)) < epsilon:
        converged = True

    iterations += 1
    if iterations % 1000 == 0:
        print(np.sum(np.fabs(V - V_)))

f'# Iterations {iterations} | Time {process_time() - start:.4f}'

'# Iterations 21 | Time 0.0058'

#### Value Function

In [100]:
print(pd.DataFrame(V.reshape(grid_size)))

          0         1         2         3
0  0.884143  0.925054  0.961986  0.000000
1  0.848181  0.000000  0.714643  0.000000
2  0.808345  0.773328  0.736099  0.516083


In [107]:
np.allclose(V.reshape(grid_size), np.asarray(vi.V).reshape(grid_size))

True

#### Optimal Policy

In [110]:
for state, reward in absorbing_states.items():
    V[state] = reward

policy = np.argmax(np.sum(transitions * V, 2),0)
policy

array([2, 2, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0])

In [112]:
pd.DataFrame(policy.reshape(grid_size)).replace(dict(enumerate(actions)))

,0,1,2,3
0,R,R,R,L
1,U,L,L,L
2,U,L,L,L


### Policy Iteration

In [121]:
def policy_improvement(value, transitions):
    for state, reward in absorbing_states.items():
        value[state] = reward
    return np.argmax(np.sum(transitions * value, 2),0)

In [123]:
V = np.random.rand(num_states)

V[skip_states] = 0

pi = np.random.choice(list(range(num_actions)), size=num_states)

In [125]:
iterations = 0

start = process_time()

converged = False

while not converged:
    pi_ = np.copy(pi)
    for state in states_to_update:
        action = policy[state]
        V[state] = np.dot(transitions[action, state], (rewards[action, state] + gamma* V))
        pi = policy_improvement(V.copy(), transitions)
    if np.array_equal(pi_, pi):
        converged = True
    iterations += 1

f'# Iterations {iterations} | Time {process_time() - start:.4f}'

'# Iterations 3 | Time 0.0071'

In [127]:
pd.DataFrame(pi.reshape(grid_size)).replace(dict(enumerate(actions)))

,0,1,2,3
0,R,R,R,L
1,U,L,U,L
2,U,L,U,L


In [129]:
pd.DataFrame(V.reshape(grid_size))

,0,1,2,3
0,0.779379,0.911291,0.956407,0.000000
1,0.704524,0.000000,0.640517,0.000000
2,0.629705,0.565474,0.533483,0.327995
